# Aula 7: 

In [1]:
import numpy as np
import sympy as sp

from scipy.linalg import expm
from IPython.display import display, Math
from  scipy.special import factorial

In [2]:
class ObjQuantico:
    def __init__(self, data,latex_representation=None):
        self.dados = data
        self.latex_representation = latex_representation

    def definir_dados(self, data):
        self.dados = data

    def full(self):
        return self.dados
    
    def dim(self):
        return len(self.dados)
    
    def dag(self):
        return ObjQuantico(np.conjugate(self.dados.T))
    
    def traço(self):
        return np.trace(self.dados).real
    
    def Autovalores(self):
        return np.linalg.eigvals(self.dados)
    
    def Autovetores(self):
        return np.linalg.eig(self.dados)[1]
    
    def AutoValor_vetor(self):
        return np.linalg.eig(self.dados)[1]
    
    def expM(self):
        return ObjQuantico(expm(self.dados)) 

    def __repr__(self):
        if self.latex_representation:
            display(Math(self.latex_representation))
        else:
            display(Math(sp.latex(sp.Matrix(self.dados))))
        return f"ObjQuantico: dim ={self.dim()} , shape = {self.dados.shape}" 
    
    def __mul__(self, other):
        # Multiplicação para diferentes tipos
        if isinstance(other, ObjQuantico):  
            # Multiplicação matricial com outra instância de ObjQuantico
            return ObjQuantico(np.dot(self.dados, other.dados))
        elif np.isscalar(other):  # Multiplicação por escalar
            return ObjQuantico(self.dados * other)
        else:
            raise TypeError(f"Multiplicação não suportada entre {type(other)} e ObjQuantico")

    def __rmul__(self, other):
        if np.isscalar(other):  # Multiplicação reversa por escalar
            return ObjQuantico(self.dados * other)
        else:
            raise TypeError(f"Multiplicação não suportada entre {type(other)} e ObjQuantico")
        
    def __sub__(self, other):
        if isinstance(other, ObjQuantico):  
            # Subtração entre duas instâncias de ObjQuantico
            return ObjQuantico(self.dados - other.dados)
        else:
            raise TypeError(f"Subtração não suportada entre {type(other)} e ObjQuantico")
       
    def __add__(self, other):
        if isinstance(other, ObjQuantico):  
            # Soma os dados de dois objetos ObjQuantico
            return ObjQuantico(self.dados + other.dados)
        else:
            raise TypeError(f"Soma não suportada entre {type(other)} e ObjQuantico")
        
    def __sub__(self, other):
        if isinstance(other, ObjQuantico):  
            # Subtração entre duas instâncias de ObjQuantico
            return ObjQuantico(self.dados - other.dados)
        elif isinstance(other, np.ndarray):  # Subtração com arrays NumPy
            return ObjQuantico(self.dados - other)
        else:
            raise TypeError(f"Subtração não suportada entre {type(other)} e ObjQuantico")
    
    def __rsub__(self, other):
        if isinstance(other, np.ndarray):  # Subtração com arrays NumPy (comutada)
            return ObjQuantico(other - self.dados)
        else:
            raise TypeError(f"Subtração não suportada entre {type(other)} e ObjQuantico")   

    def __truediv__(self, other):
        if isinstance(other, (int, float)):  # Divisão por um número escalar
            return ObjQuantico(self.dados / other)
        else:
            raise TypeError(f"Divisão não suportada entre {type(other)} e ObjQuantico")
    
    def __rtruediv__(self, other):
        if isinstance(other, (int, float)):  # Divisão invertida por um número escalar
            return ObjQuantico(other / self.dados)
        else:
            raise TypeError(f"Divisão não suportada entre {type(other)} e ObjQuantico")     
    
    def __matmul__(self, other):
        """Implementa o operador @ para o produto tensorial."""
        if isinstance(other, ObjQuantico):
            return ObjQuantico(np.kron(self.full(), other.full()))
        else:
            raise TypeError(f"Operador @ não suportado entre {type(self)} e {type(other)}")

def bases(N,n):
    estadoinicial = np.zeros(shape=(N, 1))
    estadoinicial[n, 0] = 1
    return ObjQuantico(estadoinicial) 
  
def ket(entrada):
    if isinstance(entrada, str):
        if entrada == '0':
            dados = np.array([[1], [0]])
            latex_representation = r"$$ \ket{0} $$"  # LaTeX para o ket |0>
            return ObjQuantico(dados, latex_representation)
        elif entrada == '1':
            dados = np.array([[0], [1]])
            latex_representation = r"$$ \ket{1} $$"  # LaTeX para o ket |1>
            return ObjQuantico(dados, latex_representation)
    else:
        try:
            return ObjQuantico(entrada)
        except ValueError:
            return print("Entrada invalida.") 
         
def bra(entrada):
    if isinstance(entrada, str):
        if entrada == '0':
            dados = np.array([[1], [0]])
            latex_representation = r"$$ \bra{0} $$"  # LaTeX para o ket |0>
            return ObjQuantico(dados, latex_representation)
        elif entrada == '1':
            dados = np.array([[0], [1]])
            latex_representation = r"$$ \bra{1} $$"  # LaTeX para o ket |0>
            return ObjQuantico(dados, latex_representation)
    else:
        try:
            return ObjQuantico(entrada)
        except ValueError:
            return print("Entrada invalida.")      

def destruiçao(N):
    subdiag = np.sqrt(np.arange(1, N))# Monta os elementos na subdiagonal
    dt      = np.diag(subdiag, k=1) # Operador de destruição
    return ObjQuantico(dt)

def criaçao(N):
    return  destruiçao(N).dag()    
   
def Identidade(N):
    matriz = np.identity(N)
    return ObjQuantico(matriz) 

def pauliX():
    m = np.array([[ 0, 1 ],[ 1, 0 ]])
    latex_representation = r"$$ \hat{\sigma_x} $$"
    return ObjQuantico(m,latex_representation)

def pauliY():
    m = np.array([[ 0, -1j ],[ 1j, 0 ]])
    latex_representation = r"$$ \hat{\sigma_y} $$"  
    return ObjQuantico(m,latex_representation)

def pauliZ():
    m = np.array([[ 1, 0 ],[ 0, -1 ]])
    latex_representation = r"$$ \hat{\sigma_z} $$"  
    return ObjQuantico(m,latex_representation)

def Fock(N, n=0):
    "Equivalente a função bases"
    return bases(N, n)

def coerente(N,alpha,metodo ="operador"):
    if metodo == "operador" :
        estado  = bases(N,0) # estado inicinal no vacuo
        D       = alpha * destruiçao(N).dag() - np.conj(alpha) * destruiçao(N)
        D       = D.expM()
        return D*estado
    
    elif metodo == "analitico":    
        estado  = np.zeros(shape=(N,1),dtype=complex)
        n       = np.arange(N)
        estado[:,0] = np.exp(-(abs(alpha) ** 2 )/ 2.0) * (alpha**n)/np.sqrt(factorial(n))
        return estado
    else:
        raise TypeError(
            "A opção de método tem as seguintes opções :'operador' ou 'analitico'")
        

# Solver With scipy

*   solve_ivp(fun, t_span, y0[, method, t_eval, ...]):

**    Solve an initial value problem for a system of ODEs.

*   RK45(fun, t0, y0, t_bound[, max_step, rtol, ...]):

**    Explicit Runge-Kutta method of order 5(4).

*   DOP853(fun, t0, y0, t_bound[, max_step, ...]):

**  Explicit Runge-Kutta method of order 8.

*   Radau(fun, t0, y0, t_bound[, max_step, ...])

**  Implicit Runge-Kutta method of Radau IIA family of order 5.

*   LSODA(fun, t0, y0, t_bound[, first_step, ...])

**  Adams/BDF method with automatic stiffness detection and switching.